### Comp Screen with Custom Formula

In [23]:
import sys
import os
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config


config_info = {
    "canalyst_api_key":"",
   "s3_access_key_id": "",
    "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)



## Select our ticker list

In [15]:
ticker_list = "V US,MA US,PYPL US"

## Create a Modelset, establish a feature list of time series, and create a dataframe

In [16]:
dashboard_modelset = cd.ModelSet(ticker_list,config=config,allow_nulls = True)
feature_list = ['MO_VA_EV',
                'MO_VA_EV_ToEBITDA',
                'MO_VA_EV_ToSales',
               'MO_RIS_EPS_WAD',
                'MO_VA_StockPrice']

df = dashboard_modelset.model_frame(time_series_name=feature_list,
                                    period_duration_type='fiscal_year',
                                    is_historical=True)

## The data returns in "long" or "tidy" form

In [17]:
df.head()

,ticker,period_name,period_duration_type,period_start_date,period_end_date,category,category_type_slug,time_series_slug,time_series_name,category_type_name,...,unit_description,unit_symbol,unit_type,value,is_historical,MRFQ,period_name_sorted,CSIN,is_driver,name_index
3162,V US,FY2009,fiscal_year,2008-10-01,2009-09-30,Valuation,financial-analysis-category-type,enterprise-value-avg,MO_VA_EV,Financial Analysis,...,USD,$,currency,38030741250.00,True,Q1-2022,2009FY,2CRCA60110,False,507.00
129208,PYPL US,FY2009,fiscal_year,2009-01-01,2009-12-31,Valuation,financial-analysis-category-type,enterprise-value-avg,MO_VA_EV,Financial Analysis,...,USD,$,currency,NaN,True,Q4-2021,2009FY,EQKXBS0187,False,477.00
86490,MA US,FY2009,fiscal_year,2009-01-01,2009-12-31,Valuation,financial-analysis-category-type,enterprise-value-avg,MO_VA_EV,Financial Analysis,...,USD,$,currency,21256900000.00,True,Q4-2021,2009FY,GSYQ3D0168,False,1128.00
3163,V US,FY2010,fiscal_year,2009-10-01,2010-09-30,Valuation,financial-analysis-category-type,enterprise-value-avg,MO_VA_EV,Financial Analysis,...,USD,$,currency,53302827500.00,True,Q1-2022,2010FY,2CRCA60110,False,507.00
129209,PYPL US,FY2010,fiscal_year,2010-01-01,2010-12-31,Valuation,financial-analysis-category-type,enterprise-value-avg,MO_VA_EV,Financial Analysis,...,USD,$,currency,NaN,True,Q4-2021,2010FY,EQKXBS0187,False,477.00


## Use time_series_formula to create a custom time series

In [20]:
formula_params = {'arguments' : ['MO_VA_StockPrice','MO_RIS_EPS_WAD'],
                 'modifier' : 'divide',
                 'time_series_name' : 'PE_Ratio',
                 'time_series_description' : 'PE Ratio'}


dashboard_modelset.time_series_formula(**formula_params) ##The model_frame() is modified in place

## Now establish our new feature list, including our custom time series, 
## and pivot long to wide
## = Comp Screen!

In [22]:
feature_list = ['MO_VA_EV',
                'MO_VA_EV_ToEBITDA',
                'MO_VA_EV_ToSales',
               'MO_RIS_EPS_WAD',
                'PE_Ratio']
df = dashboard_modelset.model_frame(time_series_name=feature_list,
                                    period_duration_type='fiscal_year',
                                    is_historical=True)
df.pivot(index = ['ticker','time_series_name'],columns = 'period_name',values = 'value')

period_name                       FY2009         FY2010         FY2011  \
ticker  time_series_name                                                 
MA US   MO_RIS_EPS_WAD              1.12           1.41           1.48   
        MO_VA_EV          21256900000.00 25964640000.00 33857344000.00   
        MO_VA_EV_ToEBITDA           8.54           8.76           9.01   
        PE_Ratio                   16.81          16.38          20.36   
PYPL US MO_RIS_EPS_WAD               NaN            NaN            NaN   
        MO_VA_EV                     NaN            NaN            NaN   
        MO_VA_EV_ToEBITDA            NaN            NaN            NaN   
        MO_VA_EV_ToSales             NaN            NaN            NaN   
        PE_Ratio                     NaN            NaN            NaN   
V US    MO_RIS_EPS_WAD              0.77           1.00           1.29   
        MO_VA_EV          38030741250.00 53302827500.00 50177023750.00   
        MO_VA_EV_ToEBITDA           9.80          10.79           8.50   
        PE_Ratio                   18.69          19.90          15.13   

period_name                       FY2012          FY2013          FY2014  \
ticker  time_series_name                                                   
MA US   MO_RIS_EPS_WAD              2.19            2.56            3.09   
        MO_VA_EV          47524210000.00  68169681250.00  85084170000.00   
        MO_VA_EV_ToEBITDA          11.35           13.86           15.72   
        PE_Ratio                   19.03           23.89           24.86   
PYPL US MO_RIS_EPS_WAD               NaN            0.78            0.34   
        MO_VA_EV                     NaN             NaN             NaN   
        MO_VA_EV_ToEBITDA            NaN             NaN             NaN   
        MO_VA_EV_ToSales             NaN             NaN             NaN   
        PE_Ratio                     NaN            0.00            0.00   
V US    MO_RIS_EPS_WAD              0.79            1.90            2.16   
        MO_VA_EV          69534242500.00 106875840000.00 130158541250.00   
        MO_VA_EV_ToEBITDA          10.35           13.67           14.86   
        PE_Ratio                   35.50           21.91           24.57   

period_name                        FY2015          FY2016          FY2017  \
ticker  time_series_name                                                    
MA US   MO_RIS_EPS_WAD               3.35            3.69            3.65   
        MO_VA_EV           98747611250.00 101377167500.00 133575140000.00   
        MO_VA_EV_ToEBITDA           17.87           16.09           18.01   
        PE_Ratio                    26.83           25.75           34.71   
PYPL US MO_RIS_EPS_WAD               1.00            1.15            1.47   
        MO_VA_EV           37815195000.00  40281560000.00  60677571250.00   
        MO_VA_EV_ToEBITDA           15.14           14.57           17.69   
        MO_VA_EV_ToSales       4089013.30      3715325.59      4633998.11   
        PE_Ratio                    35.48           33.40           37.51   
V US    MO_RIS_EPS_WAD               2.58            2.48            2.80   
        MO_VA_EV          154820160000.00 197538907500.00 226870481250.00   
        MO_VA_EV_ToEBITDA           15.86           18.84           17.51   
        PE_Ratio                    25.19           30.48           31.93   

period_name                        FY2018          FY2019          FY2020  \
ticker  time_series_name                                                    
MA US   MO_RIS_EPS_WAD               5.60            7.94            6.37   
        MO_VA_EV          197795803224.30 262742668333.33 312846485849.80   
        MO_VA_EV_ToEBITDA           21.82           25.18           34.81   
        PE_Ratio                    34.19           32.26           48.56   
PYPL US MO_RIS_EPS_WAD               1.71            2.07            3.54   
        MO_VA_EV           91660113545.82 11861353000